configure mysql/python connection and prepare the databse for use

In [1]:
# import password from config file
from config import db_password

In [2]:
# import mysql.connector
import mysql.connector as connector

In [5]:
# connect MySQL with Python
connection = connector.connect(user='root', password = db_password)
print("Connection between MySQL and Python is established.\n")

Connection between MySQL and Python is established.



In [6]:
# create cursor
cursor = connection.cursor()
print("Cursor is created to communicate with the MySQL using Python.\n")

Cursor is created to communicate with the MySQL using Python.



In [7]:
# use the database
cursor.execute('USE little_lemon_db;')
print("The database little_lemon is set for use.\n")

The database little_lemon is set for use.



ESTABLISH POOL CONNECTION 

In [8]:
# import pool connection and Error from mysql.connector.pooling
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

In [9]:
# configure dbconfig
dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": db_password
}

try:
    pool =MySQLConnectionPool(pool_name='pool_a', pool_size=2, **dbconfig)
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The connection pool size is: ', pool.pool_size)

except Error as er:
    print('Error: ', er.errno)
    print('Message: ', er.msg)

The connection pool is created with a name:  pool_a
The connection pool size is:  2


In [10]:
# get connection object from a pool
print('\nGetting connection from a pool ...\n')
connection1 = pool.get_connection()
print("'Connection1' object is: ", connection1)


Getting connection from a pool ...

'Connection1' object is:  <mysql.connector.pooling.PooledMySQLConnection object at 0x105eed040>


In [11]:
# create cursor object from a connection
cursor = connection1.cursor()

CREATE PROCEDURES

In [12]:
# CREATE PROCEDURE PeakHours
cursor.execute('DROP PROCEDURE IF EXISTS PeakHours;')
stored_procedure_peak_hours = ''' CREATE PROCEDURE PeakHours()
BEGIN
SELECT HOUR(BookingSlot) AS Booking_Hour, COUNT(HOUR(BookingSlot)) AS n_Bookings
FROM Bookings
GROUP BY HOUR(BookingSlot)
ORDER BY n_Bookings DESC;
END
'''


In [13]:
# execute the stored procedure
cursor.execute(stored_procedure_peak_hours)

In [14]:
# call the stored procedure
cursor.callproc('PeakHours')

# retrieve records in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# retrieve the column names
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# print the column names and dataset
print(columns)

#print the dataset
for row in dataset:
    print(row)


['Booking_Hour', 'n_Bookings']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


stored procedure GuestStatus

In [18]:
# create procedure GuestStatus
cursor.execute('DROP PROCEDURE IF EXISTS GuestStatus;')
stored_procedure_guest_status = ''' CREATE PROCEDURE GuestStatus()
BEGIN 
SELECT CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS 'Guest Name', 
e.Role AS Employee,

CASE 
WHEN e.Role IN ('Manager', 'Assistant Manager') THEN 'Ready to pay'
WHEN e.Role = 'Head Chef' THEN 'Ready to serve'
WHEN e.Role = 'Assistant Chef' THEN 'Preparing Order' 
WHEN e.Role = 'Head Waiter' THEN 'Order Served' 
ELSE 'Pending'
END AS Status

FROM Bookings AS b
LEFT JOIN Employees AS e
ON b.EmployeeID = e.EmployeeID;

END

'''

In [19]:
# execute the stored procedure
cursor.execute(stored_procedure_guest_status)


In [20]:
# call the stored procedure
cursor.callproc('GuestStatus')

# retrieve records in dataset
results = next(cursor.stored_results())
dataset = results.fetchall()

# retrieve the column names
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

# print the column names and dataset
print(columns)

#print the dataset
for row in dataset:
    print(row)

# close connection
connection1.close()


['Guest Name', 'Employee', 'Status']
('Anna Iversen', 'Manager', 'Ready to pay')
('Joakim Iversen', 'Manager', 'Ready to pay')
('Vanessa McCarthy', 'Head Chef', 'Ready to serve')
('Marcos Romero', 'Assistant Chef', 'Preparing Order')
('Hiroki Yamane', 'Assistant Manager', 'Ready to pay')
('Diana Pinto', 'Head Waiter', 'Order Served')


create connection for pool_b

In [21]:
# configure connection for pool_b using try-except block
try:
    pool = MySQLConnectionPool(pool_name='pool_b', pool_size=2, **dbconfig)
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The connection pool size is: ', pool.pool_size)

except Error as er:
    print('Error: ', er.errno)
    print('Message: ', er.msg)


The connection pool is created with a name:  pool_b
The connection pool size is:  2
